In [7]:
import re
import os
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import json
import time
from collections import Counter
from tqdm import tqdm

In [8]:
df = pd.read_csv("../../synthetic-original.csv")

df

#from sklearn.model_selection import train_test_split

#train, test = train_test_split(df, test_size=0.2)

,UserID,Docs,Action,Summaries
0,U335175,"['N41340', 'N55476', 'N103556', 'N27570', 'N83...","['click', 'click', 'click', 'click', 'click', ...",2
1,U146053,"['N95202', 'N84319', 'N92854', 'N43563', 'S3',...","['click', 'click', 'skip', 'gen_summ', 'summ_g...",4
2,U158889,"['N84182', 'N72110', 'N122127', 'N24095', 'N96...","['skip', 'skip', 'skip', 'skip', 'skip', 'skip...",0
3,U22232,"['N25386', 'N90820', 'N35729', 'N32113', 'N444...","['click', 'skip', 'click', 'click', 'skip', 'c...",1
4,U32515,"['N55509', 'N111634', 'N15992', 'N78883', 'N27...","['click', 'skip', 'click', 'click', 'click', '...",1
...,...,...,...,...
399989,U199504,"['N43145', 'N93930', 'N28736', 'N52394', 'N549...","['click', 'skip', 'skip', 'skip', 'click', 'cl...",2
399990,U236246,"['N37463', 'N53716', 'N84319', 'N20284', 'N224...","['click', 'click', 'click', 'click', 'skip', '...",2
399991,U318251,"['N91695', 'N38780', 'N78552', 'N52505', 'N876...","['skip', 'click', 'skip', 'click', 'click', 's...",1
399992,U215323,"['N47207', 'N15248', 'N50002', 'N62450', 'N479...","['click', 'click', 'skip', 'click', 'click', '...",6


In [9]:
train = df

In [16]:
def extract_click_news_ids(df):
    clicknewsID_dict = {}
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        user_id = row['UserID']
        actions = eval(row['Action'])
        docs = eval(row['Docs'])
        
        if user_id not in clicknewsID_dict:
            clicknewsID_dict[user_id] = []
        
        for i, action in enumerate(actions):
            if action == 'click':
                # print(clicknewsID_dict[user_id])
                clicknewsID_dict[user_id].append(docs[i])
    
    # Convert lists to comma-separated strings
    for user_id in clicknewsID_dict:
        clicknewsID_dict[user_id] = ' '.join(clicknewsID_dict[user_id])
    
    return clicknewsID_dict



def extract_pos_news_ids(df):
    posnewsID_dict = {}
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        user_id = row['UserID']
        actions = eval(row['Action'])
        docs = eval(row['Docs'])
        
        if user_id not in posnewsID_dict:
            posnewsID_dict[user_id] = []
        
        for i, action in enumerate(actions):
            # Include both gen_summ and summ_gen actions
            if action in ['gen_summ', 'summ_gen']:
                posnewsID_dict[user_id].append(docs[i])
    
    # Convert lists to comma-separated strings
    for user_id in posnewsID_dict:
        posnewsID_dict[user_id] = ' '.join(posnewsID_dict[user_id])
    
    return posnewsID_dict

In [13]:
pos_newsID = extract_pos_news_ids(train)


100%|██████████| 399994/399994 [01:34<00:00, 4249.56it/s]


In [17]:
click_newsID = extract_click_news_ids(train)

100%|██████████| 399994/399994 [01:33<00:00, 4277.89it/s]


In [34]:
def extract_news_ids(df, action_type):
    news_ids_dict = {}
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        user_id = row['UserID']
        actions = eval(row['Actions'])
        docs = eval(row['Docs'])
        
        if user_id not in news_ids_dict:
            news_ids_dict[user_id] = []
        
        for i, action in enumerate(actions):
            if action == action_type:
                news_ids_dict[user_id].append(docs[i])
    
    # Convert lists to comma-separated strings
    for user_id in news_ids_dict:
        news_ids_dict[user_id] = ' '.join(news_ids_dict[user_id])
    
    return news_ids_dict

# Assuming df is your DataFrame containing 'UserID', 'Actions', and 'Docs' columns
# Example: df = pd.read_csv('your_data.csv')

# Extract news IDs based on different actions
clicknewsID_dict = extract_news_ids(train, 'click')
posnewsID_dict = extract_news_ids(train, 'gen_summ')
negnewsID_dict = extract_news_ids(train, 'skip')

# Modified version of the final combination code
all_user_ids = train['UserID']  # Get all unique user IDs

combined_data = {
    'UserID': list(all_user_ids),
    'clicknewsID': [clicknewsID_dict.get(user_id, '') for user_id in all_user_ids],
    'pos': [posnewsID_dict.get(user_id, '') for user_id in all_user_ids],
    'neg': [negnewsID_dict.get(user_id, '') for user_id in all_user_ids]
}

trainer_df = pd.DataFrame(combined_data)

# Save the combined DataFrame to a CSV file
trainer_df.to_csv('trainer_aug-openai_50_500.csv', index=False)

# Display the resulting DataFrame
trainer_df

100%|██████████| 1280/1280 [00:01<00:00, 1112.72it/s]


,UserID,clicknewsID,pos,neg
0,OKFDIsAZl6Qa0m9x26f5Ao4S0uc7Ca,t3_1uik1u t3_43w7kx t3_4ekgl7 t3_211gkj t3_30m...,t3_35gz3l t3_4fplod t3_rjg7p t3_2u0li8 t3_1ivr...,t3_lzpxj t3_119rvc t3_ameih t3_1yrlga t3_3fxc6...
1,4voZkkCJyOCnpQ5f8WFf5unLW1dSjC,t3_3tlgya t3_2mpjax t3_33e2n9 t3_1nethz t3_3o6...,t3_2fv0ml t3_1k899m t3_2zhdyd t3_3nxhny t3_3hc...,t3_3ioudp t3_3rsxjl t3_33ypga t3_500g0r t3_19w...
2,BYcMzzjuFgaA59QDKoAgY07PyyG0qC,t3_40cfd4 t3_2xzm0r t3_1c29x1 t3_1c4076 t3_3ed...,t3_1i2p7t t3_o2vo1 t3_4pe3ee t3_3afl3k t3_2ckw...,t3_211gkj t3_hkbuc t3_3p7ucm t3_167gcz t3_3afl...
3,3AFaFd3w9NjDGnO51kupLyK1N44DQ2,t3_2jkqnb t3_29092z t3_3mtzaq t3_2ggb1u t3_1ng...,t3_35eusg t3_2m6xw7 t3_3167r6 t3_hdtc0 t3_39dt...,t3_398tyr t3_2eqmlc t3_3w3zcj t3_3gdg9m t3_n6d...
4,sC4a4UNRMSYCGopXr3K8znnyna6TVh,t3_2yk9rt t3_2e8wr5 t3_3yxzgv t3_2k0qfz t3_3yx...,t3_hk8rk t3_2k0qfz t3_1bx4k6 t3_25vak1 t3_ckog...,t3_17nvxc t3_1bx4k6 t3_2kauk9 t3_3yxzgv t3_2k0...
...,...,...,...,...
1275,isDhsCwFEZSpFQj0r8qHU3SpdkWtpw,t3_1xzlz1 t3_wwd84 t3_1reivt t3_3et50u t3_i4b8...,t3_2qaz0c t3_20c1hv t3_37c0dd t3_sz09b t3_2nsf...,t3_xc1d1 t3_3adqvs t3_37c0dd t3_1jesoc t3_1cd1...
1276,ZzGCcAhvqF0HnKxNsUjtJFadcZdyZj,t3_3ya4an t3_4xi5u6 t3_3c3rd8 t3_3toaal t3_43j...,t3_1bv6d5 t3_ozlc8 t3_1nrtmr t3_363wpl t3_rirf...,t3_vadsk t3_4e471z t3_3a3r92 t3_2ihb43 t3_2uhp...
1277,P2p07Up4eJyvxrrVYgwtb60krbFbxI,t3_288uwq t3_gy5aj t3_187d6t t3_3k1lcg t3_nsub...,t3_10rvhi t3_1bl2rb t3_t7rpj t3_39kvu8 t3_54c1...,t3_2xfozg t3_3clvmw t3_o1dv6 t3_24za8p t3_17cb...
1278,a7zXgbkuY6lk3vdt0q2Qf7SrZQgZ86,t3_qfryk t3_20tqos t3_4q8lgg t3_rbz4x t3_2xtys...,t3_1krjmn t3_4k1e9z t3_2u7v30 t3_3vy8xu t3_n1t...,t3_u1uhi t3_24q6bu t3_2szuhx t3_2jzljg t3_4pxx...
